<a href="https://colab.research.google.com/github/fred-ykv/Value-Investing-In-Python/blob/master/NEW_FINVIZ_REVISAO_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar bibliotecas necessárias
!pip install yfinance
!pip install git+https://github.com/quantopian/pyfolio

  Cloning https://github.com/quantopian/pyfolio to /tmp/pip-req-build-vfq_nupp
  Running command git clone --filter=blob:none --quiet https://github.com/quantopian/pyfolio /tmp/pip-req-build-vfq_nupp
  Resolved https://github.com/quantopian/pyfolio to commit 4b901f6d73aa02ceb6d04b7d83502e5c6f2e81aa
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import seaborn as sns
from urllib.request import Request, urlopen

pd.set_option('display.max_colwidth', 25)



In [ ]:
# Importar as bibliotecas
import pyfolio as pf
import warnings
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen
import yfinance as yf

# Ignorar warnings
warnings.filterwarnings('ignore')

In [ ]:
# Solicitar o ticker ao usuário
symbol = input('Enter a ticker: ')
print(f'Getting data for {symbol}...\n')

Enter a ticker: mli
Getting data for mli...



In [ ]:
# Extrair dados do Finviz
url = f"http://finviz.com/quote.ashx?t={symbol.lower()}"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

# Checar resposta
print(req)

In [ ]:
# Adicionar dados financeiros do Yahoo Finance
print("\nFetching additional data from Yahoo Finance...\n")
try:
    # Baixar os dados usando o método atualizado do yfinance
    stock_data = yf.download(symbol, start="2020-01-01", end="2023-01-01")
    print(f"Data from Yahoo Finance for {symbol}:\n")
    print(stock_data.head())
except Exception as e:
    print(f"Error fetching data from Yahoo Finance: {e}")


Fetching additional data from Yahoo Finance...

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Data from Yahoo Finance for mli:

Price           Close       High        Low       Open  Volume
Ticker            MLI        MLI        MLI        MLI     MLI
Date                                                          
2020-01-02  14.694951  14.843385  14.578987  14.801638  297600
2020-01-03  14.690312  14.699590  14.439830  14.453746  421200
2020-01-06  14.532600  14.625372  14.500131  14.611456  318400
2020-01-07  14.425915  14.551156  14.356337  14.514048  259200
2020-01-08  14.379530  14.509410  14.356338  14.393446  209800


In [ ]:
def get_insider():
    try:
        # Find insider table
        insider = pd.read_html(str(html), attrs = {'class': 'body-table'})[0]

        # Clean up insider dataframe
        insider = insider.iloc[1:]
        insider.columns = ['Trader', 'Relationship', 'Date', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']
        insider = insider[['Date', 'Trader', 'Relationship', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']]
        insider = insider.set_index('Date')
        return insider

    except Exception as e:
        return e

print('\nRecent Insider Trades: ')
insider = get_insider()
if insider is not None:
    print(insider)
else:
    print("Recent insider trades not found.")


Recent Insider Trades: 
                              Trader              Relationship  \
Date                                                             
May 12 '25             HANSEN JOHN B                   Officer   
May 12 '25           Sigloch Steffen  Chief Manufacturing O...   
May 12 '25             HANSEN JOHN B                  Director   
May 06 '25           HERMANSON TERRY                  Director   
Apr 30 '25          GLADSTEIN GARY S                  Director   
Apr 29 '25         GOLDMAN SCOTT JAY                  Director   
Apr 03 '25           Sigloch Steffen  Chief Manufacturing O...   
Mar 10 '25           Sigloch Steffen  Chief Manufacturing O...   
Mar 11 '25           Sigloch Steffen  Chief Manufacturing O...   
Feb 28 '25           Sigloch Steffen  Chief Manufacturing O...   
Feb 28 '25           Steffen Sigloch                   Officer   
Feb 13 '25         GOLDMAN SCOTT JAY                  Director   
Feb 06 '25         GOLDMAN SCOTT JAY               

In [ ]:
def get_news():
    try:
        # Find news table
        news = pd.read_html(str(html), attrs = {'class': 'fullview-news-outer'})[0]
        links = []
        for a in html.find_all('a', class_="tab-link-news"):
            links.append(a['href'])

        # Clean up news dataframe
        news.columns = ['Date', 'News Headline']
        news['Article Link'] = links
        news = news.set_index('Date')
        return news

    except Exception as e:
        return e

print ('\nRecent News: ')
print(get_news())



Recent News: 
                              News Headline              Article Link
Date                                                                 
May-09-25 01:52PM  Mueller Industries, I...  https://www.businessw...
06:18AM            Mueller Industries, I...  /news/53036/mueller-i...
May-03-25 10:39PM  Top 50 High-Quality D...  https://seekingalpha....
08:30AM            5 Relatively Secure A...  https://seekingalpha....
Apr-29-25 05:40PM  L.B. Foster Company: ...  https://seekingalpha....
...                                     ...                       ...
Mar-12-23 08:02AM  Mueller Industries, I...  https://finance.yahoo...
Feb-22-23 01:54AM  6 big dividends: Home...  https://finance.yahoo...
Feb-21-23 07:00AM  Mueller Industries, I...  https://finance.yahoo...
Feb-17-23 06:35PM  Purple Innovation, On...  https://finance.yahoo...
06:35PM            Purple Innovation and...  https://finance.yahoo...

[100 rows x 2 columns]


In [ ]:
pesquisa = html.find_all("table")
print(pesquisa[9])

<table border="0" cellpadding="3" cellspacing="0" class="js-snapshot-table snapshot-table2 screener_snapshot-table-body" width="100%">
<tr class="table-dark-row">
<td align="left" class="snapshot-td2 cursor-pointer w-[7%]" data-boxover="cssbody=[tooltip_short_bdy] cssheader=[tooltip_short_hdr] body=[Major index membership] offsetx=[10] offsety=[20] delay=[300]">Index</td><td align="left" class="snapshot-td2 w-[8%]" style=""><b>RUT</b></td>
<td align="left" class="snapshot-td2 cursor-pointer w-[7%]" data-boxover="cssbody=[tooltip_short_bdy] cssheader=[tooltip_short_hdr] body=[Price-to-Earnings (ttm)] offsetx=[10] offsety=[20] delay=[300]">P/E</td><td align="left" class="snapshot-td2 w-[8%]" style=""><b><span class="color-text is-positive">14.42</span></b></td>
<td align="left" class="snapshot-td2 cursor-pointer w-[7%]" data-boxover="cssbody=[tooltip_short_bdy] cssheader=[tooltip_short_hdr] body=[Diluted EPS (ttm)] offsetx=[10] offsety=[20] delay=[300]">EPS (ttm)</td><td align="left" cla

In [ ]:
# Extrai os atributos desejados da tabela (assumindo que o índice da tabela é 9)
table = html.find_all("table")[9]
rows = table.find_all("tr")

atributos = []
for row in rows:
    # Extraia células de dados e limpe o conteúdo do texto
    data_cells = [cell.text.strip() for cell in row.find_all("td")]
    atributos.append(data_cells)

# Imprima ou use a lista de atributos extraída (considere uma possível limpeza de dados)
print(atributos)




[['Index', 'RUT', 'P/E', '14.42', 'EPS (ttm)', '5.49', 'Insider Own', '3.48%', 'Shs Outstand', '110.65M', 'Perf Week', '5.72%'], ['Market Cap', '8.76B', 'Forward P/E', '11.55', 'EPS next Y', '6.85', 'Insider Trans', '-3.42%', 'Shs Float', '106.80M', 'Perf Month', '7.62%'], ['Income', '623.95M', 'PEG', '-', 'EPS next Q', '1.62', 'Inst Own', '92.55%', 'Short Float', '3.81%', 'Perf Quarter', '-1.85%'], ['Sales', '3.92B', 'P/S', '2.23', 'EPS this Y', '12.99%', 'Inst Trans', '3.64%', 'Short Ratio', '4.34', 'Perf Half Y', '-17.00%'], ['Book/sh', '24.10', 'P/B', '3.28', 'EPS next Y', '14.17%', 'ROA', '20.37%', 'Short Interest', '4.07M', 'Perf Year', '35.80%'], ['Cash/sh', '8.00', 'P/C', '9.89', 'EPS next 5Y', '-', 'ROE', '24.51%', '52W Range', '53.53 - 96.81', 'Perf YTD', '-0.29%'], ['Dividend Est.', '-', 'P/FCF', '17.32', 'EPS past 5Y', '42.73%', 'ROI', '23.19%', '52W High', '-18.27%', 'Beta', '1.04'], ['Dividend TTM', '0.85 (1.07%)', 'Quick Ratio', '3.24', 'Sales past 5Y', '11.51%', 'Gross 

In [ ]:
# Inicializa listas vazias para armazenar atributos e valores
attributes = []
values = []

# Iterar sobre cada linha (excluindo a linha do cabeçalho)
for row in rows:
    data_cells = [cell.text.strip() for cell in row.find_all("td")]

    # Se a linha não estiver vazia
    if len(data_cells) > 0:
        # Extrai atributos e valores de cada célula de dados
        for i, cell_text in enumerate(data_cells):
            if i % 2 == 0:  # Índices pares são atributos
                attributes.append(cell_text)
            else:  # Índices ímpares são valores
                values.append(cell_text)

# Crie um DataFrame a partir dos atributos e valores extraídos
df = pd.DataFrame({'Atributos': attributes, 'Valores': values})

# Configurar o Pandas para exibir todas as linhas e colunas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Imprimir o DataFrame
print(df)

           Atributos        Valores
0              Index            RUT
1                P/E          14.42
2          EPS (ttm)           5.49
3        Insider Own          3.48%
4       Shs Outstand        110.65M
5          Perf Week          5.72%
6         Market Cap          8.76B
7        Forward P/E          11.55
8         EPS next Y           6.85
9      Insider Trans         -3.42%
10         Shs Float        106.80M
11        Perf Month          7.62%
12            Income        623.95M
13               PEG              -
14        EPS next Q           1.62
15          Inst Own         92.55%
16       Short Float          3.81%
17      Perf Quarter         -1.85%
18             Sales          3.92B
19               P/S           2.23
20        EPS this Y         12.99%
21        Inst Trans          3.64%
22       Short Ratio           4.34
23       Perf Half Y        -17.00%
24           Book/sh          24.10
25               P/B           3.28
26        EPS next Y        

**Valor de Mercado**

In [ ]:
# Encontrar o índice do atributo desejado
market_cap_str = values[attributes.index('Market Cap')]

# Se o último caractere for 'B', multiplique por 1 bilhão
# Se o último caractere for 'M', multiplique por 1 milhão
# Caso contrário, mantenha o valor como está
if market_cap_str[-1] == 'B':
    market_cap = float(market_cap_str[:-1]) * 1000000000
elif market_cap_str[-1] == 'M':
    market_cap = float(market_cap_str[:-1]) * 1000000
else:
    market_cap = float(market_cap_str)

# Imprimir o valor do Market Cap formatado
print(f'Market Cap: ${market_cap:,.2f}')


Market Cap: $8,760,000,000.00


**ROI**

In [ ]:
# Encontrar o índice do atributo ROI
roi_str = values[attributes.index('ROI')]

# Remover o '%' do final e converter para float, dividindo por 100 para obter o valor percentual
roi_perc_str = float(roi_str[:-1]) / 100

# Imprimir o ROI
print(f'ROI: {roi_perc_str}')



ROI: 0.23190000000000002


**ROE**

In [ ]:
# Encontrar o índice do atributo ROE
roe_str = values[attributes.index('ROE')]

# Remover o '%' do final e converter para float, dividindo por 100 para obter o valor percentual
roe_perc_str = float(roi_str[:-1]) / 100

# Imprimir o ROE
print(f'ROE: {roe_perc_str}')


ROE: 0.23190000000000002


**PAYOUT**

In [ ]:
# Encontrar o índice do atributo PAYOUT
payout_str = values[attributes.index('Payout')]

# Remover o '%' do final e converter para float, dividindo por 100 para obter o valor percentual
payout_perc_str = float(payout_str[:-1]) / 100

# Imprimir o ROE
print(f'PAYOUT: {payout_perc_str}')


PAYOUT: 0.1507


**DIVIDEND YIELD**

In [ ]:
# Encontrar o índice do atributo Dividend Est.
div_str = values[attributes.index('Dividend Est.')]

# Verificar se o valor é '-' e tratá-lo como 0
if div_str == '-':
    div_total_value = 0
else:
    # Apagar os valores fora dos parênteses
    div_value = div_str.split('(')[-1].split(')')[0]

    # Remover os parênteses
    div_value = div_value.replace('(', '').replace(')', '')

    # Retirar o símbolo '%'
    div_value = div_value.replace('%', '')

    # Transformar em float
    div_float = float(div_value)

    # Dividir o valor por 100
    div_total_value = div_float / 100

# Imprimir o Dividend Est. renomeado para DIVIDEND YIELD
print(f'DIVIDEND YIELD: {div_total_value}')



DIVIDEND YIELD: 0


**Crescimento dos Lucros nos próximos 5 anos**

In [ ]:
# Encontrar o índice do atributo EPS next 5Y
epsn5_index = attributes.index('EPS next 5Y')

# Verificar se o valor de EPS next 5Y é '-' e tratá-lo como 0, e se há múltiplos valores para o atributo 'EPS next Y'
if epsn5_index == '-':
    # Verificar se há múltiplos valores para o atributo 'EPS next Y'
    epsn_y_indices = [i for i, attr in enumerate(attributes) if attr == 'EPS next Y']

    if len(epsn_y_indices) >= 2:
        # Pegar o segundo índice da lista
        epsn_y_index = epsn_y_indices[1]

        # Pegar o valor correspondente a esse índice
        epsn_y_str = values[epsn_y_index]
    else:
        epsn_y_str = '-'

    # Verificar se o valor de EPS next Y é '-' e tratá-lo como 0
    if epsn_y_str == '-':
        epsn5_perc_str = 0
else:
    epsn5_str = values[epsn5_index]
    # Verificar se o valor é '-' e tratá-lo como 0
    if epsn5_str == '-':
        epsn5_perc_str = 0
    else:
        # Remover o '%' do final e converter para float, dividindo por 100 para obter o valor percentual
        epsn5_perc_str = float(epsn5_str[:-1]) / 100

# Imprimir o EPS next 5Y
print(f'EPS next 5Y: {epsn5_perc_str}')






EPS next 5Y: 0


**LPA - Lucro por Ação**

In [ ]:
# Encontrar o índice do atributo LPA
eps_str = values[attributes.index('EPS (ttm)')]

# Transformar o VP/A em float
eps_float = float(eps_str)

# Imprimir o VP/A (Valor partrimônial por ação)
print(f'LPA: {eps_float}')

LPA: 5.49


**VPA - Valor Patrimonial por Ação**

In [ ]:
# Encontrar o índice do atributo VP/A
VPA_str = values[attributes.index('Book/sh')]

# Transformar o VP/A em float
VPA_float = float(VPA_str)

# Imprimir o VP/A (Valor partrimônial por ação)
print(f'VPA: {VPA_float}')

VPA: 24.1


**BETA**

In [ ]:
# Encontrar o índice do atributo Beta
beta_str = values[attributes.index('Beta')]

# Transformar o Beta em float
beta_float = float(beta_str)

# Imprimir o Beta
print(f'Beta: {beta_float}')


Beta: 1.04


**Taxa de Reinvestimento do Lucro Líquido (bLL)**

In [ ]:
# Definir a taxa de reinvestimento com base na diferença entre 1 e o percentual de pagamento (payout)
tax_reinvest_ll = 1 - payout_perc_str

# Imprimir a taxa de reinvestimento
print(f'Taxa de Reinvestimento bLL: {tax_reinvest_ll:.3f}')


Taxa de Reinvestimento bLL: 0.849


**gLL - Crescimento do Lucro Líquido**

In [ ]:
gll = tax_reinvest_ll*roe_perc_str


print(f'gLL: {gll:.3f}')

gLL: 0.197


**Ganho de Alavancagem**

In [ ]:
gain_leverage = roe_perc_str - roi_perc_str


print(f'Ganho de Alavancagem: {gain_leverage:.3f}')

Ganho de Alavancagem: 0.000


**Grau de Alavancagem Financeira**

In [ ]:
gaf = roe_perc_str/roi_perc_str


print(f'Grau de Alavancagem Financeira: {gaf:.2f}')

Grau de Alavancagem Financeira: 1.00


**Taxa de Juros Livre de Risco (Rf) - Treasury Bond 10Y**

In [ ]:
# Requisição Url
url = "https://ycharts.com/indicators/10_year_treasury_rate"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Parser
html = soup(webpage, "html.parser")

# Buscando Valor
xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    Rf = float(value.replace(',', '.'))
    Rf_str = str(Rf) + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {Rf_str}')

Taxa de Juros Livre de Risco (Rf): 4.53%


In [ ]:
url = 'https://ycharts.com/indicators/us_10year_government_bond_interest_rate'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    interest_rate = float(value.replace(',', '.'))
    interest_rate_str = str(interest_rate) + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {interest_rate_str}')

Taxa de Juros Livre de Risco (Rf): 4.32%


**Taxa de Inflação Americana**

In [ ]:
# Requisição Url
url = "https://ycharts.com/indicators/us_consumer_price_index_yoy"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Parser
html = soup(webpage, "html.parser")

# Buscando Valor
xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    inflation = float(value.replace(',', '.'))
    inflation_str = str(inflation) + '%' # Corrigindo Erro

    print(f'Taxa de Inflação Americana: {inflation_str}')

Taxa de Inflação Americana: 2.31%


**S&P 500 - Retorno 10Y**

In [ ]:
# Definir função para calcular o retorno
def calcula_retorno(dataframe):
    df = dataframe.copy()
    df['daily_return'] = df.iloc[:, 0].pct_change()
    df['acum_return'] = (1 + df['daily_return']).cumprod()
    df.iloc[0, df.columns.get_loc('acum_return')] = 1  # Corrigir o valor inicial
    df.rename(columns={df.columns[0]: "price"}, inplace=True)
    return df

# Baixar dados do S&P 500
sp500_data = yf.download('^GSPC')

# Verificar se 'Adj Close' existe, senão usar 'Close'
if 'Adj Close' in sp500_data.columns:
    sp500 = sp500_data[['Adj Close']]
else:
    sp500 = sp500_data[['Close']]

# Calcular os retornos
sp500 = calcula_retorno(sp500)

# Calcular o retorno acumulado em 10 anos e o retorno médio anual
Ten_year_return = sp500['acum_return'].iloc[-1] * 100 - 100
mean_yearly_return = sp500['daily_return'].mean() * 100 * 365

# Melhorar a visualização
print(f'\nRetorno acumulado em 10 anos: {Ten_year_return:.2f}%')
print(f'Retorno médio anual em 10 anos: {mean_yearly_return:.2f}%')


[*********************100%***********************]  1 of 1 completed



Retorno acumulado em 10 anos: 33439.75%
Retorno médio anual em 10 anos: 11.29%


**Taxa de Inflação Brasil**

In [ ]:
url = 'https://br.investing.com/economic-calendar/brazilian-cpi-410'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

xpath = '/html/body/div[5]/section/div[6]/span[2]/div'
value_elem = html.find('div', {'class':'inner_cell'})

if value_elem is not None:
    value = value_elem.get_text().strip()

    # Removendo Caracteres
    taxa_juros_br = float(value.replace(',', '.'))
    taxa_juros_br_str = str(taxa_juros_br).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {taxa_juros_br_str}')

# **INDICADORES**

In [ ]:
# Criar um novo DataFrame com as respostas organizadas
answers = pd.DataFrame({
    'Indicadores': ['Market Cap', 'ROI','ROE','Payout','Dividend %','EPS next 5Y','LPA','VPA','Beta','Taxa Reinvestimento bLL','gLL','Ganho de Alavancagem','Grau Alavancagem Finac','Tx Juros Livre de Risco','Taxa de Inflação USA','Retorno acum 10Y SP500','Retorno médio 10Y SP500'],
    'Valor': [f'${market_cap:,.2f}', f' {roi_perc_str:.3f}',f' {roe_perc_str:.3f}',f' {payout_perc_str:.3f}',f' {div_total_value}',f' {epsn5_perc_str}',f' {eps_float}',f' {VPA_float}',f' {beta_str}',f' {tax_reinvest_ll:.3f}',f' {gll:.3f}',f' {gain_leverage:.3f}',f' {gaf:.3f}',f' {Rf_str}',f' {inflation_str}',f' {Ten_year_return:.1f}%',f' {mean_yearly_return:.2f}%']
})

print(answers)

                Indicadores              Valor
0                Market Cap  $8,760,000,000.00
1                       ROI              0.232
2                       ROE              0.232
3                    Payout              0.151
4                Dividend %                  0
5               EPS next 5Y                  0
6                       LPA               5.49
7                       VPA               24.1
8                      Beta               1.04
9   Taxa Reinvestimento bLL              0.849
10                      gLL              0.197
11     Ganho de Alavancagem              0.000
12   Grau Alavancagem Finac              1.000
13  Tx Juros Livre de Risco              4.53%
14     Taxa de Inflação USA              2.31%
15   Retorno acum 10Y SP500           33439.8%
16  Retorno médio 10Y SP500             11.29%


# **Balanço Patrimonial**

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/'+ symbol +'/balance-sheet'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

**Patrimônio Líquido (PL)**

In [ ]:
# Implementação
try:
    # 1 - Encontrando todas as ocorrências de 'Total Shareholder's Equity'
    equity_rows = soup.find_all('td', text="Total Shareholder's Equity")
    if not equity_rows:
        raise ValueError("Linhas 'Total Shareholder's Equity' não encontradas.")

    # Selecionar a última ocorrência
    last_equity_row = equity_rows[-1]

    # Obtendo os valores da linha
    values = last_equity_row.find_next_siblings('td')
    values_text = [val.text.strip() for val in values]

    # 2 - Verificar se o primeiro valor é 'NA'
    if values_text[0] == 'NA':
        # Usar o quarto valor da direita para a esquerda
        value = values_text[-4]
    else:
        value = values_text[0]

    # 3 - Removendo as vírgulas e convertendo o valor para float
    value_in_millions = float(value.replace(',', '')) * 1_000_000

    # 4 - Formatando o valor de volta para uma string com a notação de dólares
    value_formatted = '${:,.2f}'.format(value_in_millions)

    # 5 - Imprimindo o valor formatado
    print('Patrimônio Líquido (PL):', value_formatted)

except Exception as e:
    print("Erro:", e)


Erro: Linhas 'Total Shareholder's Equity' não encontradas.


**Empréstimos e Financiamentos**

In [ ]:
# Implementação
try:
    # 1 - Encontrando a linha 'Current Portion Long-Term Debt'
    current_portion_longterm_debt = soup.find('td', text="Current Portion Long-Term Debt")
    if not current_portion_longterm_debt:
        raise ValueError("Linha 'Current Portion Long-Term Debt' não encontrada.")

    # Obtendo os valores da linha
    values = current_portion_longterm_debt.find_next_siblings('td')
    values_text = [val.text.strip() for val in values]

    # 2 - Verificar se o primeiro valor é 'NA'
    if values_text[0] == 'NA':
        # Usar o quarto valor da direita para a esquerda
        value = values_text[-4]
    else:
        value = values_text[0]

    # 3 - Removendo as vírgulas e convertendo o valor para float
    value_in_millions = float(value.replace(',', '')) * 1_000_000

    # 4 - Formatando o valor de volta para uma string com a notação de dólares
    value_formatted = '${:,.2f}'.format(value_in_millions) if value_in_millions != 0 else '$ 0.00'

    # 5 - Imprimindo o valor formatado
    print('Empréstimos e Financiamentos:', value_formatted)

except Exception as e:
    print("Erro:", e)


Erro: Linha 'Current Portion Long-Term Debt' não encontrada.


**Passivo Total**

In [ ]:
# Implementação
try:
    # 1 - Encontrando a linha 'Total Liabilities'
    total_liabilities = soup.find('td', text="Total Liabilities")
    if not total_liabilities:
        raise ValueError("Linha 'Total Liabilities' não encontrada.")

    # Obtendo os valores da linha
    values = total_liabilities.find_next_siblings('td')
    values_text = [val.text.strip() for val in values]

    # 2 - Verificar se o primeiro valor é 'NA'
    if values_text[0] == 'NA':
        # Usar o quarto valor da direita para a esquerda
        value = values_text[-4]
    else:
        value = values_text[0]

    # 3 - Removendo as vírgulas e convertendo o valor para float
    value_in_millions_15 = float(value.replace(',', '')) * 1_000_000

    # 4 - Formatando o valor de volta para uma string com a notação de dólares
    value_formatted = '${:,.2f}'.format(value_in_millions_15) if value_in_millions_15 != 0 else '$ 0.00'

    # 5 - Imprimindo o valor formatado
    print('Passivo Total:', value_formatted)

except Exception as e:
    print("Erro:", e)


Erro: Linha 'Total Liabilities' não encontrada.


**Dívida de Longo Prazo**

In [ ]:
# Implementação
try:
    # 1 - Encontrando a linha 'Long-Term Debt'
    longterm_debt = soup.find('td', text="Long-Term Debt")
    if not longterm_debt:
        raise ValueError("Linha 'Long-Term Debt' não encontrada.")

    # Obtendo os valores da linha
    values = longterm_debt.find_next_siblings('td')
    values_text = [val.text.strip() for val in values]

    # 2 - Verificar se o primeiro valor é 'NA'
    if values_text[0] == 'NA':
        # Usar o quarto valor da direita para a esquerda
        value = values_text[-4]
    else:
        value = values_text[0]

    # 3 - Removendo as vírgulas e convertendo o valor para float
    value_in_millions = float(value.replace(',', '')) * 1_000_000

    # 4 - Formatando o valor de volta para uma string com a notação de dólares
    value_formatted = '${:,.2f}'.format(value_in_millions) if value_in_millions != 0 else '$ 0.00'

    # 5 - Imprimindo o valor formatado
    print('Dívida de Longo Prazo:', value_formatted)

except Exception as e:
    print("Erro:", e)


Erro: Linha 'Long-Term Debt' não encontrada.


**Capital de Terceiros (Po)**

In [ ]:
try:
    # 1 - Encontrando a linha 'Current Portion Long-Term Debt'
    current_portion_longterm_debt_row = soup.find('td', text="Current Portion Long-Term Debt")
    if not current_portion_longterm_debt_row:
        raise ValueError("Linha 'Current Portion Long-Term Debt' não encontrada.")

    # Obtendo os valores da linha
    current_portion_values = current_portion_longterm_debt_row.find_next_siblings('td')
    current_portion_text = [val.text.strip() for val in current_portion_values]

    # Verificar se o primeiro valor é 'NA'
    if current_portion_text[0] == 'NA':
        current_portion_value = current_portion_text[-4]
    else:
        current_portion_value = current_portion_text[0]

    # Removendo as vírgulas e convertendo o valor para float
    current_portion_in_millions = float(current_portion_value.replace(',', '')) * 1_000_000

    # 2 - Encontrando a linha 'Long-Term Debt'
    longterm_debt_row = soup.find('td', text="Long-Term Debt")
    if not longterm_debt_row:
        raise ValueError("Linha 'Long-Term Debt' não encontrada.")

    # Obtendo os valores da linha
    longterm_values = longterm_debt_row.find_next_siblings('td')
    longterm_text = [val.text.strip() for val in longterm_values]

    # Verificar se o primeiro valor é 'NA'
    if longterm_text[0] == 'NA':
        longterm_value = longterm_text[-4]
    else:
        longterm_value = longterm_text[0]

    # Removendo as vírgulas e convertendo o valor para float
    longterm_in_millions = float(longterm_value.replace(',', '')) * 1_000_000

    # 3 - Somando os valores
    total_debt = current_portion_in_millions + longterm_in_millions

    # 4 - Formatando o valor total
    total_debt_formatted = '${:,.2f}'.format(total_debt) if total_debt != 0 else '$ 0.00'

    # 5 - Imprimindo o valor formatado
    print('Capital de Terceiros (PO):', total_debt_formatted)

except Exception as e:
    print("Erro:", e)

Erro: Linha 'Current Portion Long-Term Debt' não encontrada.


**Capital Investido (Ativos Totais)**

In [ ]:
try:
    # 1 - Encontrando a linha 'Total Assets'
    total_assets_row = soup.find('td', text="Total Assets")
    if not total_assets_row:
        raise ValueError("Linha 'Total Assets' não encontrada.")

    # Obtendo os valores da linha
    total_assets_values = total_assets_row.find_next_siblings('td')
    total_assets_text = [val.text.strip() for val in total_assets_values]

    # Verificar se o primeiro valor é 'NA'
    if total_assets_text[0] == 'NA':
        total_assets_value = total_assets_text[-4]
    else:
        total_assets_value = total_assets_text[0]

    # 2 - Removendo as vírgulas e convertendo o valor para float
    total_assets_in_millions = float(total_assets_value.replace(',', '')) * 1_000_000

    # 3 - Formatando o valor total
    total_assets_formatted = '${:,.2f}'.format(total_assets_in_millions) if total_assets_in_millions != 0 else '$ 0.00'

    # 4 - Imprimindo o valor formatado
    print('Capital Investido (Ativos Totais):', total_assets_formatted)

except Exception as e:
    print("Erro:", e)

Erro: Linha 'Total Assets' não encontrada.


**Caixa e Aplicações Financeiras**

In [ ]:
try:
    # 1 - Encontrando a linha 'Cash & Equivalents'
    cash_equivalents_row = soup.find('td', text="Cash & Equivalents")
    if not cash_equivalents_row:
        raise ValueError("Linha 'Cash & Equivalents' não encontrada.")

    # Obtendo os valores da linha
    cash_equivalents_values = cash_equivalents_row.find_next_siblings('td')
    cash_equivalents_text = [val.text.strip() for val in cash_equivalents_values]

    # 2 - Verificar condições específicas: 'NA' ou '-99,999'
    if cash_equivalents_text[0] in ('NA', '-99,999'):
        cash_equivalents_value = cash_equivalents_text[-4]  # Usar o quarto valor da direita
    else:
        cash_equivalents_value = cash_equivalents_text[0]

    # 3 - Removendo as vírgulas e convertendo o valor para float
    cash_equivalents_in_millions = float(cash_equivalents_value.replace(',', '')) * 1_000_000

    # 4 - Formatando o valor
    cash_equivalents_formatted = '${:,.2f}'.format(cash_equivalents_in_millions) if cash_equivalents_in_millions != 0 else '$ 0.00'

    # 5 - Imprimindo o valor formatado
    print('Caixa e Aplicações Financeiras:', cash_equivalents_formatted)

except Exception as e:
    print("Erro:", e)


Erro: Linha 'Cash & Equivalents' não encontrada.


**Ativo Circulante**

In [ ]:
try:
    # 1 - Encontrando a linha 'Total Current Assets'
    total_current_assets_row = soup.find('td', text="Total Current Assets")
    if not total_current_assets_row:
        raise ValueError("Linha 'Total Current Assets' não encontrada.")

    # Obtendo os valores da linha
    total_current_assets_values = total_current_assets_row.find_next_siblings('td')
    total_current_assets_text = [val.text.strip() for val in total_current_assets_values]

    # 2 - Verificar condições específicas: 'NA' ou '-99,999'
    if total_current_assets_text[0] in ('NA', '-99,999'):
        total_current_assets_value = total_current_assets_text[-4]  # Usar o quarto valor da direita
    else:
        total_current_assets_value = total_current_assets_text[0]

    # 3 - Removendo as vírgulas e convertendo o valor para float
    total_current_assets_in_millions = float(total_current_assets_value.replace(',', '')) * 1_000_000

    # 4 - Formatando o valor
    total_current_assets_formatted = (
        '${:,.2f}'.format(total_current_assets_in_millions)
        if total_current_assets_in_millions != 0
        else '$ 0.00'
    )

    # 5 - Imprimindo o valor formatado
    print('Ativo Circulante:', total_current_assets_formatted)

except Exception as e:
    print("Erro:", e)


Erro: Linha 'Total Current Assets' não encontrada.


**Ativo Circulante Operacional**

In [ ]:
try:
    # 1 - Encontrando a linha 'Total Current Assets'
    total_current_assets_row = soup.find('td', text="Total Current Assets")
    if not total_current_assets_row:
        raise ValueError("Linha 'Total Current Assets' não encontrada.")

    total_current_assets_values = total_current_assets_row.find_next_siblings('td')
    total_current_assets_text = [val.text.strip() for val in total_current_assets_values]

    # 2 - Condição para 'Total Current Assets': 'NA' ou '-99,999'
    if total_current_assets_text[0] in ('NA', '-99,999'):
        total_current_assets_value = total_current_assets_text[-4]  # Usar o quarto valor da direita
    else:
        total_current_assets_value = total_current_assets_text[0]

    # 3 - Encontrando a linha 'Cash & Equivalents'
    cash_equivalents_row = soup.find('td', text="Cash & Equivalents")
    if not cash_equivalents_row:
        raise ValueError("Linha 'Cash & Equivalents' não encontrada.")

    cash_equivalents_values = cash_equivalents_row.find_next_siblings('td')
    cash_equivalents_text = [val.text.strip() for val in cash_equivalents_values]

    # 4 - Condição para 'Cash & Equivalents': 'NA' ou '-99,999'
    if cash_equivalents_text[0] in ('NA', '-99,999'):
        cash_equivalents_value = cash_equivalents_text[-4]  # Usar o quarto valor da direita
    else:
        cash_equivalents_value = cash_equivalents_text[0]

    # 5 - Removendo as vírgulas e convertendo para float
    total_current_assets_in_millions = float(total_current_assets_value.replace(',', '')) * 1_000_000
    cash_equivalents_in_millions = float(cash_equivalents_value.replace(',', '')) * 1_000_000

    # 6 - Calculando o Ativo Circulante Operacional
    ativo_circulante_operacional = total_current_assets_in_millions - cash_equivalents_in_millions

    # 7 - Formatando o valor para exibição
    ativo_circulante_operacional_formatted = (
        '${:,.2f}'.format(ativo_circulante_operacional)
        if ativo_circulante_operacional != 0
        else '$ 0.00'
    )

    # 8 - Imprimindo o valor formatado
    print('Ativo Circulante Operacional:', ativo_circulante_operacional_formatted)

except Exception as e:
    print("Erro:", e)


Erro: Linha 'Total Current Assets' não encontrada.


In [ ]:
# 1º Passo
# Encontrando a linha de Total Current Assets
total_current_assets_row = soup.find('td', text="Total Current Assets").find_parent('tr')

# Extraindo todos os valores da linha
values_assets = [td.text.strip() for td in total_current_assets_row.find_all('td')]

# Imprimindo os valores da linha
print("Total Current Assets Row:", values_assets)

# 2º Passo
# Encontrando a linha de Cash & Equivalents
cash_and_equivalents_row = soup.find('td', text="Cash & Equivalents").find_parent('tr')

# Extraindo todos os valores da linha
values_cash_and_equivalents = [td.text.strip() for td in cash_and_equivalents_row.find_all('td')]

# Imprimindo os valores da linha
print("Cash & Equivalents Row:", values_cash_and_equivalents)

AttributeError: 'NoneType' object has no attribute 'find_parent'

**Ativo Circulante Operacional - Ano anterior**

In [ ]:
try:
    # 1 - Encontrando a linha 'Total Current Assets'
    total_current_assets_row = soup.find('td', text="Total Current Assets")
    if not total_current_assets_row:
        raise ValueError("Linha 'Total Current Assets' não encontrada.")

    total_current_assets_values = total_current_assets_row.find_next_siblings('td')
    total_current_assets_text = [val.text.strip() for val in total_current_assets_values]

    # 2 - Condição para 'Total Current Assets': 'NA' ou '-99,999'
    if total_current_assets_text[0] in ('NA', '-99,999'):
        total_current_assets_value = total_current_assets_text[-4]  # Usar o quarto valor da direita
    else:
        total_current_assets_value = total_current_assets_text[0]

    # 3 - Encontrando a linha 'Cash & Equivalents'
    cash_equivalents_row = soup.find('td', text="Cash & Equivalents")
    if not cash_equivalents_row:
        raise ValueError("Linha 'Cash & Equivalents' não encontrada.")

    cash_equivalents_values = cash_equivalents_row.find_next_siblings('td')
    cash_equivalents_text = [val.text.strip() for val in cash_equivalents_values]

    # 4 - Condição para 'Cash & Equivalents': 'NA' ou '-99,999'
    if cash_equivalents_text[0] in ('NA', '-99,999'):
        cash_equivalents_value = cash_equivalents_text[-4]  # Usar o quarto valor da direita
    else:
        cash_equivalents_value = cash_equivalents_text[0]

    # 5 - Removendo as vírgulas e convertendo para float
    total_current_assets_in_millions = float(total_current_assets_value.replace(',', '')) * 1_000_000
    cash_equivalents_in_millions = float(cash_equivalents_value.replace(',', '')) * 1_000_000

    # 6 - Calculando o Ativo Circulante Operacional
    ativo_circulante_operacional = total_current_assets_in_millions - cash_equivalents_in_millions

    # 7 - Formatando o valor para exibição
    ativo_circulante_operacional_formatted = (
        '${:,.2f}'.format(ativo_circulante_operacional)
        if ativo_circulante_operacional != 0
        else '$ 0.00'
    )

    # 8 - Imprimindo o valor formatado
    print('Ativo Circulante Operacional:', ativo_circulante_operacional_formatted)

except Exception as e:
    print("Erro:", e)




**Passivo Circulante**

In [ ]:
try:
    # 1 - Encontrando o valor de 'Total Current Liabilities'
    total_current_liabilities = soup.find('td', text="Total Current Liabilities")
    if not total_current_liabilities:
        raise ValueError("'Total Current Liabilities' não encontrado.")

    value_text = total_current_liabilities.find_next('td').text.strip()

    # 2 - Tratando condições especiais para valores inválidos
    if value_text in ('NA', '-99,999'):
        # Usar o quarto valor da direita se os dados forem inválidos
        sibling_values = total_current_liabilities.find_next_siblings('td')
        value_text = sibling_values[-4].text.strip()

    # 3 - Removendo as vírgulas e convertendo o valor para float
    value_in_millions = float(value_text.replace(',', '')) * 1_000_000

    # 4 - Formatando o valor para exibição
    value_formatted = (
        '${:,.2f}'.format(value_in_millions)
        if value_in_millions != 0
        else '$ 0.00'
    )

    # 5 - Imprimindo o valor formatado
    print('Passivo Circulante:', value_formatted)

except Exception as e:
    print("Erro ao processar o valor de 'Total Current Liabilities':", e)


**Passivo Circulante Operacional**

In [ ]:
try:
    # Encontrando o valor de Total Current Liabilities
    total_current_liabilities = soup.find('td', text="Total Current Liabilities")
    if not total_current_liabilities:
        raise ValueError("'Total Current Liabilities' não encontrado.")

    value_liabilities = total_current_liabilities.find_next('td').text.strip()

    # Tratando condições especiais para valores inválidos de 'Total Current Liabilities'
    if value_liabilities in ('NA', '-99,999'):
        sibling_values = total_current_liabilities.find_next_siblings('td')
        value_liabilities = sibling_values[-4].text.strip()

    # Encontrando o valor de Current Portion Long-Term Debt
    current_portion_longterm_debt = soup.find('td', text="Current Portion Long-Term Debt")
    if not current_portion_longterm_debt:
        raise ValueError("'Current Portion Long-Term Debt' não encontrado.")

    value_debt = current_portion_longterm_debt.find_next('td').text.strip()

    # Tratando condições especiais para valores inválidos de 'Current Portion Long-Term Debt'
    if value_debt in ('NA', '-99,999'):
        sibling_values_debt = current_portion_longterm_debt.find_next_siblings('td')
        value_debt = sibling_values_debt[-4].text.strip()

    # Removendo as vírgulas e convertendo os valores para float
    value_liabilities_in_millions = float(value_liabilities.replace(',', '')) * 1_000_000
    value_debt_in_millions = float(value_debt.replace(',', '')) * 1_000_000

    # Calcular o Passivo Circulante Operacional
    passivo_circulante_operacional = value_liabilities_in_millions - value_debt_in_millions

    # Formatando o valor de volta para uma string com a notação de dólares
    value_formatted = (
        '${:,.2f}'.format(passivo_circulante_operacional)
        if passivo_circulante_operacional != 0
        else '$ 0.00'
    )

    # Imprimindo o valor formatado
    print('Passivo Circulante Operacional:', value_formatted)

except Exception as e:
    print("Erro ao processar os valores:", e)


**Passivo Circulante Operacional - Ano anterior**

In [ ]:
try:
    # Encontrando os valores anuais de Total Current Liabilities
    total_current_liabilities_annual = soup.find_all('td', text="Total Current Liabilities")
    if not total_current_liabilities_annual:
        raise ValueError("'Total Current Liabilities' anual não encontrado.")

    total_current_liabilities_2_annual = total_current_liabilities_annual[0].find_next('td').find_next('td').text.strip()

    # Tratando condições especiais para valores inválidos de 'Total Current Liabilities'
    if total_current_liabilities_2_annual in ('NA', '-99,999'):
        sibling_values = total_current_liabilities_annual[0].find_next_siblings('td')
        total_current_liabilities_2_annual = sibling_values[-4].text.strip()

    # Encontrando os valores anuais de Current Portion Long-Term Debt
    current_portion_longterm_debt_annual = soup.find_all('td', text="Current Portion Long-Term Debt")
    if not current_portion_longterm_debt_annual:
        raise ValueError("'Current Portion Long-Term Debt' anual não encontrado.")

    current_portion_longterm_debt_2_annual = current_portion_longterm_debt_annual[0].find_next('td').find_next('td').text.strip()

    # Tratando condições especiais para valores inválidos de 'Current Portion Long-Term Debt'
    if current_portion_longterm_debt_2_annual in ('NA', '-99,999'):
        sibling_values_debt = current_portion_longterm_debt_annual[0].find_next_siblings('td')
        current_portion_longterm_debt_2_annual = sibling_values_debt[-4].text.strip()

    # Removendo as vírgulas e convertendo os valores para float
    value_liabilities_in_millions_annual = float(total_current_liabilities_2_annual.replace(',', '')) * 1_000_000
    value_debt_in_millions_annual = float(current_portion_longterm_debt_2_annual.replace(',', '')) * 1_000_000

    # Calcular o Passivo Circulante Operacional
    passivo_circulante_operacional_annual = value_liabilities_in_millions_annual - value_debt_in_millions_annual

    # Formatando o valor de volta para uma string com a notação de dólares
    value_formatted_annual = (
        '${:,.2f}'.format(passivo_circulante_operacional_annual)
        if passivo_circulante_operacional_annual != 0
        else '$ 0.00'
    )

    # Imprimindo os valores
    print('Passivo Circulante Operacional (Ano anterior):', value_formatted_annual)

except Exception as e:
    print("Erro ao processar os valores:", e)


#**Cash Flow Statements**




In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/'+ symbol +'/cash-flow-statements'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

**Despesa Financeira**

In [ ]:
try:
    # Encontrando o valor de Issuance (Repayment) of Debt
    issuance_repayment_of_debt = soup.find('td', text="Issuance (Repayment) of Debt")
    if not issuance_repayment_of_debt:
        raise ValueError("'Issuance (Repayment) of Debt' não encontrado.")

    value = issuance_repayment_of_debt.find_next('td').text.strip()

    # Tratando condições especiais para valores inválidos de 'Issuance (Repayment) of Debt'
    if value in ('NA', '-99,999'):
        sibling_values = issuance_repayment_of_debt.find_next_siblings('td')
        value = sibling_values[-4].text.strip()

    # Removendo as vírgulas e convertendo o valor para float
    value_in_millions = float(value.replace(',', '')) * 1_000_000

    # Formatando o valor de volta para uma string com a notação de dólares
    value_formatted = (
        '${:,.2f}'.format(value_in_millions)
        if value_in_millions != 0
        else '$ 0.00'
    )

    # Imprimindo o valor formatado
    print('Despesa Financeira (Issuance/Repayment of Debt):', value_formatted)

except Exception as e:
    print("Erro ao processar os valores:", e)


**Depreciação e Amortização**

In [ ]:
try:
    # Encontrando o valor de Depreciação/Amortização & Exaustão
    Depreciation_Amortization_Depletion = soup.find('td', text="Depreciation/Amortization & Depletion")
    if not Depreciation_Amortization_Depletion:
        raise ValueError("'Depreciation/Amortization & Depletion' não encontrado.")

    value = Depreciation_Amortization_Depletion.find_next('td').text.strip()

    # Tratando condições especiais para valores inválidos de 'Depreciation/Amortization & Depletion'
    if value in ('NA', '-99,999'):
        sibling_values = Depreciation_Amortization_Depletion.find_next_siblings('td')
        value = sibling_values[-4].text.strip()

    # Removendo as vírgulas e convertendo o valor para float
    value_in_millions_7 = float(value.replace(',', '')) * 1_000_000

    # Formatando o valor de volta para uma string com a notação de dólares
    value_formatted = (
        '${:,.2f}'.format(value_in_millions_7)
        if value_in_millions_7 != 0
        else '$ 0.00'
    )

    # Imprimindo o valor formatado
    print('Depreciação e Amortização:', value_formatted)

except Exception as e:
    print("Erro ao processar os valores:", e)


**CAPEX**

In [ ]:
# Encontrando o valor de Property & Equipment (CAPEX)
Property_Equipment = soup.find('td', text="Property & Equipment")
value = Property_Equipment.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor imediatamente após o 'NA'
    next_value = Property_Equipment.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e o símbolo negativo, caso exista
if value[0] == '-':
    capex = float(value[1:].replace(',', '')) * 1000000
else:
    capex = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if capex == 0:
    value_formatted = '$ 0.00'
else:
    value_formatted = '${:,.2f}'.format(capex)

# Imprimindo o valor formatado
print('CAPEX:', value_formatted)




**Aquisição / Alienação de Subsidiárias**

In [ ]:
# Encontrando o valor de Acquisition/Disposition of Subsidiaries
Acquisition = soup.find('td', text="Acquisition/ Disposition of Subsidiaries")
value = Acquisition.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor imediatamente após o 'NA'
    next_value = Acquisition.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e o símbolo negativo, caso exista
if value == 'NA':
    value_in_millions_9 = 0
elif value[0] == '-':
    value_in_millions_9 = float(value[1:].replace(',', '')) * 1000000
else:
    value_in_millions_9 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_9 == 0:
    value_formatted = '$ 0.00'
else:
    value_formatted = '${:,.2f}'.format(value_in_millions_9)

# Imprimindo o valor formatado
print('Aquisição / Alienação de Subsidiárias:', value_formatted)


**Investimentos**

In [ ]:
# Encontrando o valor de Investimentos
Investments = soup.find('td', text="Investments")
value = Investments.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor imediatamente após o 'NA'
    next_value = Investments.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e o símbolo negativo, caso exista
if value == 'NA':
    value_in_millions_10 = 0
elif value[0] == '-':
    value_in_millions_10 = float(value[1:].replace(',', '')) * 1000000
else:
    value_in_millions_10 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_10 == 0:
    value_formatted = '$ 0.00'
else:
    value_formatted = '${:,.2f}'.format(value_in_millions_10)

# Imprimindo o valor formatado
print('Investimentos:', value_formatted)


**Outras atividades de investimento**

In [ ]:
# Encontrando o valor de Outras atividades de investimento
Other_Investing_Activities = soup.find('td', text="Other Investing Activities")
value = Other_Investing_Activities.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor imediatamente após o 'NA'
    next_value = Other_Investing_Activities.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e o símbolo negativo, caso exista
if value == 'NA':
    Other_Investing_Activities = 0
elif value[0] == '-':
    Other_Investing_Activities = float(value[1:].replace(',', '')) * 1000000
else:
    Other_Investing_Activities = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if Other_Investing_Activities == 0:
    Other_Investing_Activities_formatted = '$ 0.00'
else:
    Other_Investing_Activities_formatted = '${:,.2f}'.format(Other_Investing_Activities)

# Imprimindo o valor formatado
print('Outras atividades de investimento:', Other_Investing_Activities_formatted)


**Fluxo de Caixa das atividades de investimentos**

In [ ]:
# Calculando FC das atividades de investimentos
FCAI = capex + value_in_millions_9 + value_in_millions_10 + Other_Investing_Activities

# Formatando o valor de volta para uma string com a notação de dólares
if FCAI == 0:
  FCAI_formatted = '$ 0.00'
else:
  FCAI_formatted = '${:,.2f}'.format(FCAI)

# Imprimindo o valor formatado
print('FCAI:', FCAI_formatted)

# **DRE**

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/'+ symbol +'/income-statement'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

**Receita**

In [ ]:
# Encontrando o valor de Sales
Sales = soup.find('td', text="Sales")
value = Sales.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor imediatamente após o 'NA'
    next_value = Sales.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
    value_formatted = '$ 0.00'
else:
    value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Receita:', value_formatted)


**Custos**

In [ ]:
# Encontrando o valor de Cost of Goods
Cost_of_Goods = soup.find('td', text="Cost Of Goods")
value = Cost_of_Goods.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor imediatamente após o 'NA'
    next_value = Cost_of_Goods.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_1 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_1 == 0:
    value_formatted = '$ 0.00'
else:
    value_formatted = '${:,.2f}'.format(value_in_millions_1)

# Imprimindo o valor formatado
print('Custos:', value_formatted)


**Resultado Bruto**

In [ ]:
# Encontrando o valor de Resultado Bruto
Resultado_Bruto = value_in_millions - value_in_millions_1
Resultado_Bruto

if Resultado_Bruto == 0:
  Resultado_Bruto_formatted = '$ 0.00'
else:
  Resultado_Bruto_formatted = '${:,.2f}'.format(Resultado_Bruto)

# Imprimindo o valor formatado
print('Resultado Bruto:', Resultado_Bruto_formatted)

**EBIT**

In [ ]:
# Encontrando o valor de Pretax Income
Pretax_Income = soup.find('td', text="Pretax Income")
value = Pretax_Income.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor imediatamente após o 'NA'
    next_value = Pretax_Income.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e convertendo o valor para float
ebit = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if ebit == 0:
    value_formatted = '$ 0.00'
else:
    value_formatted = '${:,.2f}'.format(ebit)

# Imprimindo o valor formatado
print('EBIT:', value_formatted)


**EBITDA**

In [ ]:
# Encontrando o valor de EBITDA
EBITDA = ebit + value_in_millions_7
EBITDA

if EBITDA == 0:
  EBITDA_formatted = '$ 0.00'
else:
  EBITDA_formatted = '${:,.2f}'.format(EBITDA)

# Imprimindo o valor formatado
print('EBITDA:', EBITDA_formatted)

**IR**

In [ ]:
# Encontrando o valor de Income Taxes
Income_Taxes = soup.find('td', text="Income Taxes")
value = Income_Taxes.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor imediatamente após o 'NA'
    next_value = Income_Taxes.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_3 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_3 == 0:
    value_formatted = '$ 0.00'
else:
    value_formatted = '${:,.2f}'.format(value_in_millions_3)

# Imprimindo o valor formatado
print('IR:', value_formatted)


# **NOPAT Restrito**

In [ ]:
# Encontrando o valor de NOPAT Restrito
NOPAT_Restrito = ebit - value_in_millions_3
NOPAT_Restrito

if NOPAT_Restrito == 0:
  NOPAT_Restrito_formatted = '$ 0.00'
else:
  NOPAT_Restrito_formatted = '${:,.2f}'.format(NOPAT_Restrito)

# Imprimindo o valor formatado
print('NOPAT Restrito:', NOPAT_Restrito_formatted)

**Depesa Financeira**

In [ ]:
# Encontrando o valor de Investment Gains/Losses
Investment_Gains_Losses = soup.find('td', text="Investment Gains/Losses")
value = Investment_Gains_Losses.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor imediatamente após o 'NA'
    next_value = Investment_Gains_Losses.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_4 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_4 == 0:
    value_formatted = '$ 0.00'
else:
    value_formatted = '${:,.2f}'.format(value_in_millions_4)

# Imprimindo o valor formatado
print('Despesa Financeira:', value_formatted)


**Lucro Líquido**


In [ ]:
# Encontrando o valor de Net Income (GAAP)
Net_Income_GAAP = soup.find('td', text="Net Income (GAAP)")
value = Net_Income_GAAP.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor imediatamente após o 'NA'
    next_value = Net_Income_GAAP.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_5 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_5 == 0:
    value_formatted = '$ 0.00'
else:
    value_formatted = '${:,.2f}'.format(value_in_millions_5)

# Imprimindo o valor formatado
print('Lucro Líquido:', value_formatted)


# **FCL**

**NOPAT Restrito**

In [ ]:
# Encontrando o valor de NOPAT Restrito
NOPAT_Restrito = ebit - value_in_millions_3
NOPAT_Restrito

if NOPAT_Restrito == 0:
  NOPAT_Restrito_formatted = '$ 0.00'
else:
  NOPAT_Restrito_formatted = '${:,.2f}'.format(NOPAT_Restrito)

# Imprimindo o valor formatado
print('NOPAT Restrito:', NOPAT_Restrito_formatted)

**Depreciação e Amortização**

In [ ]:
import requests
from bs4 import BeautifulSoup

symbol = 'AAPL'  # Defina o símbolo da ação ou o valor desejado

url = 'https://www.zacks.com/stock/quote/' + symbol + '/cash-flow-statements'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers=user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]

# Encontrando o valor de Depreciação e Amortização
Depreciation_Amortization_Depletion = soup.find('td', text="Depreciation/Amortization & Depletion")
value = Depreciation_Amortization_Depletion.find_next('td').text.strip()

# Verificando se o valor é 'NA'
if value == 'NA':
    # Pegar o próximo valor válido imediatamente após o 'NA'
    next_value = Depreciation_Amortization_Depletion.find_next('td').find_next('td').text.strip()
    value = next_value  # Atribuindo o novo valor

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_7 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_7 == 0:
    value_formatted = '$ 0.00'
else:
    value_formatted = '${:,.2f}'.format(value_in_millions_7)

# Imprimindo o valor formatado
print('Depreciação e Amortização:', value_formatted)


**FCO**

In [ ]:
# Calculando FCO
FCO = NOPAT_Restrito + value_in_millions_7

# Formatando o valor de volta para uma string com a notação de dólares
if FCO == 0:
  FCO_formatted = '$ 0.00'
else:
  FCO_formatted = '${:,.2f}'.format(FCO)

# Imprimindo o valor formatado
print('FCO:', FCO_formatted)

**Δ NC Giro**

In [ ]:
# Calculando NC Giro
NC_Giro_atual = ativo_circulante_operacional - passivo_circulante_operacional
NC_Giro_ano_anterior = ativo_circulante_operacional_annual - passivo_circulante_operacional_annual

Var_NC_Giro = NC_Giro_atual - NC_Giro_ano_anterior

NC_Giro_atual_formatted = '${:,.2f}'.format(NC_Giro_atual)
NC_Giro_ano_anterior_formatted = '${:,.2f}'.format(NC_Giro_ano_anterior)

# Formatando o valor de volta para uma string com a notação de dólares
if Var_NC_Giro == 0:
  Var_NC_Giro_formatted = '$ 0.00'
else:
  Var_NC_Giro_formatted = '${:,.2f}'.format(Var_NC_Giro)

# Imprimindo o valor formatado
print('NC Giro Atual:', NC_Giro_atual_formatted)
print('NC Giro Ano Anterior:', NC_Giro_ano_anterior_formatted)
print('Δ NC Giro:', Var_NC_Giro_formatted)

**CAPEX**

In [ ]:
CAPEX_formatted = '${:,.2f}'.format(capex)
print('CAPEX:', CAPEX_formatted)

**FCFF**

In [ ]:
# Calculando FCFF
if Var_NC_Giro > 0:
  FCFF = FCO - Var_NC_Giro - capex
elif Var_NC_Giro < 0:
  FCFF = FCO + Var_NC_Giro - capex

# Formatando o valor de volta para uma string com a notação de dólares
if FCFF == 0:
  FCFF_formatted = '$ 0.00'
else:
  FCFF_formatted = '${:,.2f}'.format(FCFF)

# Imprimindo o valor formatado
print('FCFF:', FCFF_formatted)

**Novas Dívidas**

In [ ]:
# Calculando Novas Dívidas
Novas_dividas = value_in_millions_9 + value_in_millions_10

# Formatando o valor de volta para uma string com a notação de dólares
if Novas_dividas == 0:
  Novas_dividas_formatted = '$ 0.00'
else:
  Novas_dividas_formatted = '${:,.2f}'.format(Novas_dividas)

# Imprimindo o valor formatado
print('Novas Dívidas:', Novas_dividas_formatted)

**IR**

In [ ]:
# Imprimindo o valor formatado
IR_formatted = '${:,.2f}'.format(value_in_millions_3)
print('IR:', IR_formatted)

**FCDA**

In [ ]:
# Calculando FCDA
FCDA = FCFF + Novas_dividas + value_in_millions_3

# Formatando o valor de volta para uma string com a notação de dólares
if FCDA == 0:
  FCDA_formatted = '$ 0.00'
else:
  FCDA_formatted = '${:,.2f}'.format(FCDA)

# Imprimindo o valor formatado
print('FCDA:', FCDA_formatted)

# **CAPM - Custo do Capital Próprio**

**Custo de Capital Próprio**

In [ ]:
# Convertendo beta_str para float
beta_float = float(beta_str)

# Convertendo mean_yearly_return para float
mean_yearly_return_float = float(mean_yearly_return)

# Calculando Ke
Ke = interest_rate + (beta_float * (mean_yearly_return_float - interest_rate))
Ke_formatted = '{:,.2f}%'.format(Ke)
print('Ke:', Ke_formatted)

**Ki (Kd) - Custo da Dívida**

In [ ]:
# Calculando Ki (Kd) - Custo da Dívida
if value_in_millions_4 == 0:
  Ki = Rf + inflation
else:
  Ki = (value_in_millions_4 / total_debt) + 1

# Formatando o valor de volta para uma string com a notação de dólares
if Ki == 0:
  Ki_formatted = ' 0.00 %'
else:
  Ki_formatted = '{:,.2f} %'.format(Ki)

# Imprimindo o valor formatado
print('Ki (Kd) - Custo da Dívida:', Ki_formatted)

**Po e PL**

In [ ]:
Po = total_debt
PL = value_in_millions_12
Po_formatted = '${:,.2f}'.format(total_debt)
PL_formatted = '${:,.2f}'.format(value_in_millions_12)
print ('PL:' , PL_formatted)
print ('Po:' , Po_formatted)

**PL / (Po+PL)**

In [ ]:
# Calculando PL / (Po+PL)
if value_in_millions_12 == 0:
  PL_Po_PL = 0.0
else:
  PL_Po_PL = value_in_millions_12 / (value_in_millions_12 + total_debt)

# Formatando o valor de volta para uma string com a notação de dólares
if PL_Po_PL == 0:
   PL_Po_PL_formatted = ' 0.0'
else:
  PL_Po_PL_formatted = '{:,.2f}'.format(PL_Po_PL)

# Imprimindo o valor formatado
print('PL / (Po+PL):', PL_Po_PL_formatted)

**Po / (Po+PL)**

In [ ]:
# Calculando Po / (Po+PL)
if total_debt == 0:
  Po_Po_PL = 0.0
else:
  Po_Po_PL = total_debt / (value_in_millions_12 + total_debt)

# Formatando o valor de volta para uma string com a notação de dólares
if Po_Po_PL == 0:
   Po_Po_PL_formatted = ' 0.0'
else:
  Po_Po_PL_formatted = '{:,.2f}'.format(Po_Po_PL)

# Imprimindo o valor formatado
print('Po / (Po+PL):', Po_Po_PL_formatted)

**WACC**

In [ ]:
# Calculando WACC
wacc = (Ke * PL_Po_PL ) + (Ki * Po_Po_PL)
wacc_formatted = '{:,.2f}%'.format(wacc)
print ('WACC:' , wacc_formatted)

**Custo de Oportunidade**

In [ ]:
# Calculando Co = PL*Ke
Co = value_in_millions_12*(Ke/100)
Co_formatted = '${:,.2f}'.format(Co)
value_in_millions_12_formated = '${:,.2f}'.format(value_in_millions_12)
print('PL:' ,value_in_millions_12_formated)
print ('Ke:' ,Ke_formatted)
print ('Co:' , Co_formatted)

**Lucro Econômico (EVA)**

In [ ]:
# Calculando LE = LL - Co
LE = value_in_millions_5 - Co
LE_formatted = '${:,.2f}'.format(LE)
LL_formatted = '${:,.2f}'.format(value_in_millions_5)

# Ganho em Excesso = LL - LE
GE = value_in_millions_5 - LE
GE_formatted = '${:,.2f}'.format(GE)
print ('LL:' , LL_formatted)
print ('LE:' , LE_formatted)
print ('Ganho em Excesso:' , GE_formatted)

**ROE > KE**

In [ ]:
# Removendo o '%' da string roe_str e convertendo para float
roe_float = float(roe_str.strip('%'))

# Calculando a diferença entre ROE e Ke
ROE_KE = roe_float - Ke

# Imprimindo os resultados
print('ROE: {:.2f}%'.format(roe_float))
print('KE: {:.2f}%'.format(Ke))
print('Diferença de ROE x Ke: {:.2f}%'.format(ROE_KE))

# Verificando se ROE é maior, menor ou igual a KE
if ROE_KE > 0:
    print("SIM, ROE é maior que KE")
elif ROE_KE < 0:
    print("NÃO, ROE é menor que KE")
else:
    print("IGUAL, ROE é igual a KE")


**ROI > WACC**

In [ ]:
# Removendo o '%' da string roi_str e convertendo para float
roi_float = float(roi_str.strip('%'))

# Calculando a diferença entre ROI e WACC
ROI_WACC = roi_float - wacc

# Imprimindo os resultados
print('ROI: {:.2f}%'.format(roi_float))
print('WACC: {:.2f}%'.format(wacc))
print('Diferença de ROI x WACC: {:.2f}%'.format(ROI_WACC))

# Verificando se ROI é maior, menor ou igual a WACC
if ROI_WACC > 0:
    print("SIM, ROI é maior que WACC")
elif ROI_WACC < 0:
    print("NÃO, ROI é menor que WACC")
else:
    print("IGUAL, ROI é igual a WACC")


# **Valuation**

**Nº de Ações**

In [ ]:
# Encontrar o índice do atributo desejado
shs_outstand_str = values[attributes.index('Shs Outstand')]

# Se o último caractere for 'M', multiplique por 1 milhão
# Se for 'B', multiplique por 1 bilhão
# Caso contrário, mantenha o valor como está
if shs_outstand_str[-1] == 'M':
    shs_outstand = float(shs_outstand_str[:-1]) * 1000000
elif shs_outstand_str[-1] == 'B':
    shs_outstand = float(shs_outstand_str[:-1]) * 1000000000
else:
    shs_outstand = float(shs_outstand_str)

# Imprimir o valor de 'Shs Outstand'
print(f'Nº de Ações: {shs_outstand}')




**Crescimento dos Lucros líquidos ponderado ao Mercado - gLL + Média Mercado**

In [ ]:
#Crescimento dos Lucros líquidos ponderado ao Mercado - gLL + Média Mercado
# O correto seria somar ponderar o gll + a média do crescimento do ROE de 5Y)
gll_M = (gll + (mean_yearly_return/100))/2
print(f'gLL + Média do Mercado: {gll_M:.3f}')

**bNOPAT**

In [ ]:
# Calculando bNOPAT
#bNOPAT = (CAPEX - Amortização e Depeciação) / EBIT
bNOPAT = (capex - value_in_millions_7)/ebit

print('CAPEX:', CAPEX_formatted)
print('Depreciação e Amortização:', value_formatted)
print('EBIT:', value_formatted)


print(f'bNOPAT: {bNOPAT:.3f}')

**gNOPAT**

In [ ]:
# Calculando gNOPAT
# gNOPAT = bNOPAT * ROI
gNOPAT = bNOPAT * roi_perc_str
print(f'gNOPAT: {gNOPAT:.3f}')

**Crescimeto na perpetuidade**

In [ ]:
# Calculando Crescimeto na perpetuidade
CP = 2/100
print(f'Crescimeto na perpetuidade: {CP:.3f}')

**FCFF**

In [ ]:
FCFF
# Imprimindo o valor formatado
print('FCFF:', FCFF_formatted)

**VP**

In [ ]:
# A Taxa de desconto é calculada usando a formula (1+wacc)^n
# Ano 0
Ano_0 = 0
EBITDA_0 = 0.0
FCDE_0 = FCFF
Taxa_Desconto_0 = 0.0
VP_0 = FCDE_0

# Ano 1
Ano_1 = 1
EBITDA_1 = FCDE_0 * gll_M
FCDE_1 = FCDE_0 + EBITDA_1
Taxa_Desconto_1 = pow (1 + wacc/100, Ano_1)
VP_1 = FCDE_1/Taxa_Desconto_1

# Ano 2
Ano_2 = 2
EBITDA_2 = FCDE_1 * gll_M
FCDE_2 = FCDE_1 + EBITDA_2
Taxa_Desconto_2 = pow (1 + wacc/100, Ano_2)
VP_2 = FCDE_2/Taxa_Desconto_2

# Ano 3
Ano_3 = 3
EBITDA_3 = FCDE_2 * gll_M
FCDE_3 = FCDE_2 + EBITDA_3
Taxa_Desconto_3 = pow (1 + wacc/100, Ano_3)
VP_3 = FCDE_3/Taxa_Desconto_3

# Ano 4
Ano_4 = 4
EBITDA_4 = FCDE_3 * gll_M
FCDE_4 = FCDE_3 + EBITDA_4
Taxa_Desconto_4 = pow (1 + wacc/100, Ano_4)
VP_4 = FCDE_4/Taxa_Desconto_4

# Ano 5
Ano_5 = 5
EBITDA_5 = FCDE_4 * gll_M
FCDE_5 = FCDE_4 + EBITDA_5
Taxa_Desconto_5 = pow (1 + wacc/100, Ano_5)
VP_5 = FCDE_5/Taxa_Desconto_5

# Ano 6
Ano_6 = 6
EBITDA_6 = FCDE_5 * gll_M
FCDE_6 = FCDE_5 + EBITDA_6
Taxa_Desconto_6 = pow (1 + wacc/100, Ano_6)
VP_6 = FCDE_6/Taxa_Desconto_6

# Ano 7
Ano_7 = 7
EBITDA_7 = FCDE_6 * gll_M
FCDE_7 = FCDE_6 + EBITDA_7
Taxa_Desconto_7 = pow (1 + wacc/100, Ano_7)
VP_7 = FCDE_7/Taxa_Desconto_7

# Ano 8
Ano_8 = 8
EBITDA_8 = FCDE_7 * gll_M
FCDE_8 = FCDE_7 + EBITDA_8
Taxa_Desconto_8 = pow (1 + wacc/100, Ano_8)
VP_8 = FCDE_8/Taxa_Desconto_8

# Ano 9
Ano_9 = 9
EBITDA_9 = FCDE_8 * gll_M
FCDE_9 = FCDE_8 + EBITDA_9
Taxa_Desconto_9 = pow (1 + wacc/100, Ano_9)
VP_9 = FCDE_9/Taxa_Desconto_9

# Ano 10
Ano_10 = 10
EBITDA_10 = FCDE_9 * gll_M
FCDE_10 = FCDE_9 + EBITDA_10
Taxa_Desconto_10 = pow (1 + wacc/100, Ano_10)
VP_10 = FCDE_10/Taxa_Desconto_10

# Ano 11 - Perpetuidade
Ano_11 = 11
FCDE_11 = FCDE_10
Taxa_Desconto_11 =  (wacc/100 - (gNOPAT))*100
VP_perpeturidade = (FCDE_11/(Taxa_Desconto_11)*100)
VP_11 = VP_perpeturidade / (Taxa_Desconto_10)


print("Ano  |     EBITDA         |     FCDE             |   Taxa de Desconto     |        VP")
print("{:^4} | ${:^11,.2f}       | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_0, EBITDA_0, FCDE_0, Taxa_Desconto_0, VP_0))
print("{:^4} | ${:^11,.2f}    | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_1, EBITDA_1, FCDE_1, Taxa_Desconto_1, VP_1))
print("{:^4} | ${:^11,.2f}    | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_2, EBITDA_2, FCDE_2, Taxa_Desconto_2, VP_2))
print("{:^4} | ${:^11,.2f}    | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_3, EBITDA_3, FCDE_3, Taxa_Desconto_3, VP_3))
print("{:^4} | ${:^11,.2f}    | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_4, EBITDA_4, FCDE_4, Taxa_Desconto_4, VP_4))
print("{:^4} | ${:^11,.2f}    | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_5, EBITDA_5, FCDE_5, Taxa_Desconto_5, VP_5))
print("{:^4} | ${:^11,.2f}    | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_6, EBITDA_6, FCDE_6, Taxa_Desconto_6, VP_6))
print("{:^4} | ${:^11,.2f}    | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_7, EBITDA_7, FCDE_7, Taxa_Desconto_7, VP_7))
print("{:^4} | ${:^11,.2f}    | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_8, EBITDA_8, FCDE_8, Taxa_Desconto_8, VP_8))
print("{:^4} | ${:^11,.2f}    | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_9, EBITDA_9, FCDE_9, Taxa_Desconto_9, VP_9))
print("{:^4} | ${:^11,.2f}  | ${:^11,.2f}   | {:^15.2f}        | ${:^11,.2f}".format(Ano_10, EBITDA_10, FCDE_10, Taxa_Desconto_10, VP_10))
print("{:^4}                      | ${:^11,.2f}   | {:^15.2f}        | ${:^11,.2f}".format(Ano_11, FCDE_11, Taxa_Desconto_11, VP_11))

**VP**

In [ ]:
#Encontrando o Valor Patrimonial (VP)
VP = VP_0 + VP_1 + VP_2 + VP_3 + VP_4 + VP_5 + VP_6 + VP_7 + VP_8 + VP_9 + VP_10 + VP_11
VP_formatted = '${:,.2f}'.format(VP)
print ('VP:' , VP_formatted)

# **Preço Justo da Ação**



In [ ]:
# Encontrando o Preço Justo da Ação

# Ativo Total
Ativo_Total = value_in_millions
Ativo_Total_formatted = '${:,.2f}'.format(Ativo_Total)
print(f'Ativo Total: ${Ativo_Total_formatted}')

# Passivo Total
Passivo_Total = value_in_millions_15
Passivo_Total_formatted = '${:,.2f}'.format(Passivo_Total)
print(f'Passivo Total: ${Passivo_Total_formatted}')

# PL = Ativo Total - Passivo Total
PL_ajustado = value_in_millions - value_in_millions_15
PL_ajustado_formatted = '${:,.2f}'.format(PL_ajustado)
print(f'PL Ajustado: ${PL_ajustado_formatted}')

# FCLL
print('FCFF:', FCFF_formatted)

# Número de Ações
print(f'Nº de Ações: {shs_outstand}')

# Verificar se VP é negativo
if VP < 0:
    PJA = ((value_in_millions - value_in_millions_15) + FCFF)/ shs_outstand
else:
    PJA = VP / shs_outstand

PJA_formatted = '${:,.2f}'.format(PJA)
print('PJA:', PJA_formatted)


# Calcular o preço atual
# Encontrar o índice do atributo desejado
price_str = values[attributes.index('Price')]
price = float(price_str)
# Imprimir o valor de 'Preço'
print(f'Preço: {price_str}')

Margem_P = PJA - price
Margem_P_formatted = '${:,.2f}'.format(Margem_P)
print('Margem:' , Margem_P_formatted)

Margem_Perc = (Margem_P / PJA)*100
Margem_Perc_formatted = '{:,.2f}%'.format(Margem_Perc)
print('Margem Percentual:' , Margem_Perc_formatted)

# **AVALIAÇÃO PELO LUCRO EM EXCESSO**

In [ ]:
# Calculando Vo = Capital Investido + PV do Retorno em Excesso
#PV do Retorno em Excesso = Goodwill

Goodwill = LE / Ke
Goodwill_formatted = '${:,.2f}'.format(Goodwill)
print('Goodwill',Goodwill_formatted)

# Lucros futuros esperados
LFE = gll_M * LE
LFE_formatted = '${:,.2f}'.format(LFE)
print('LFE',LFE_formatted)

# Valor da empresa = patrimônio líquido + goodwill + lucros futuros esperados
Vo_excesso = PL + (Goodwill*10) + (LFE*10)
Vo_excesso_formatted = '${:,.2f}'.format(Vo_excesso)
print('Vo',Vo_excesso_formatted)

# Valor Patrimonial por Ação
print(f'VPA: {VPA_str}')

#Encontrando o Preço Justo da Ação pelo Lucro em Excesso
PJA_E = Vo_excesso / shs_outstand
PJA_E_formatted = '${:,.2f}'.format(PJA_E)
print ('PJA pelo Lucro em Excesso:' , PJA_E_formatted)

# Calcular o preço atual
# Encontrar o índice do atributo desejado
price_str = values[attributes.index('Price')]
price = float(price_str)
# Imprimir o valor de 'Preço'
print(f'Preço: {price_str}')

# **Valor Intrínseco**

In [ ]:
import math

eps = float(eps_str)
VPA = float(VPA_str)

# Calcular o valor intrínseco em dólar
valor_intrinseco = math.sqrt(eps * VPA * 22.5)

# Calcular o preço atual
# Encontrar o índice do atributo desejado
price_str = values[attributes.index('Price')]
price = float(price_str)
# Imprimir o valor de 'Preço'
print(f'Preço: ${price_str}')

# Calcular a diferença entre o valor intrínseco e o preço atual
diferenca = valor_intrinseco - price

# Calcular a diferença percentual
dif_percentual = diferenca / valor_intrinseco * 100

# Verificar se está com ágio ou deságio
if price > valor_intrinseco:
    agio_desagio = 'ágio'
else:
    agio_desagio = 'deságio'

# Exibir os resultados
print(f"Valor intrínseco: ${valor_intrinseco:.2f}")
print(f"Diferença: ${diferenca:.2f}")
print(f"Margem percentual: {dif_percentual:.2f}% {agio_desagio}")

if agio_desagio == 'ágio':
    recomendacao = 'Recomendação: Venda'
else:
    recomendacao = 'Recomendação: Compra'

print(recomendacao)

# **Comparativo entre Avaliações**

In [ ]:
print ('Stock:',symbol)
print(f'Preço Atual: ${price_str}')
print(f"Valor intrínseco: ${valor_intrinseco:.2f}")
print ('Preço Valuation:' , PJA_formatted)
print ('PJA pelo Lucro em Excesso:' , PJA_E_formatted)

# **Retorno S&P 500 - 10 Y x Ticker em análise**

**Retorno S&P 500**

In [ ]:
Ten_year_return = sp500['acum_return'].iloc[-1] * 100 - 100
print(f'Retorno acumulado 10Y: {Ten_year_return:.1f}%')

mean_yearly_return = sp500['daily_return'].mean() * 100 * 365
print(f'Retorno médio anual 10Y: {mean_yearly_return:.2f}%')

**Retorno Ticker**

In [ ]:
def calcula_retorno(dataframe):
  df = dataframe.copy()
  df['daily_return']        = df.iloc[:,0].pct_change()
  df['acum_return']         = (1 +  df['daily_return']).cumprod()
  df['acum_return'].iloc[0] = 1
  df.rename(columns={df.columns[0]: "price" }, inplace=True)
  return df

ticker = yf.download(symbol)[['Adj Close']]
ticker

ticker = calcula_retorno(ticker)
ticker

Ten_year_return_ticker = ticker['acum_return'].iloc[-1] * 100 - 100
print(f'Retorno acumulado 10Y do Ticker: {Ten_year_return_ticker:.1f}%')

mean_yearly_return_ticker = ticker['daily_return'].mean() * 100 * 365
print(f'Retorno médio anual 10Y do Ticker: {mean_yearly_return_ticker:.2f}%')


In [ ]:

# Gráfico 1
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=sp500.index[-252*10:], y=sp500['price'][-252*10:], line=dict(color='blue')))
fig1.update_layout(yaxis_type="log")

# Adicionando o nome do ticker no canto superior esquerdo do Gráfico 1
fig1.add_annotation(
    xref="paper",
    yref="paper",
    x=0.02,
    y=0.98,
    text="SP 500",
    showarrow=False,
    font=dict(size=22)
)

fig1.show()


# Gráfico 2
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=ticker.index[-252*10:], y=ticker['price'][-252*10:], line=dict(color='red')))
fig2.update_layout(yaxis_type="log")

# Adicionando o nome do ticker no canto superior esquerdo do Gráfico 2
fig2.add_annotation(
    xref="paper",
    yref="paper",
    x=0.02,
    y=0.98,
    text="Ticker",
    showarrow=False,
    font=dict(size=22)
)

fig2.show()

**Análise correlacionadas ao Benchmarket (S&P 500)**

In [ ]:
tickers = [symbol,"^GSPC"]
dados_yahoo = web.get_data_yahoo(tickers, period="10y")["Adj Close"]

In [ ]:
dados_yahoo.columns = [symbol,"SP500"]
dados_yahoo

RETORNO DIARIOS

In [ ]:
retorno = dados_yahoo.pct_change()
retorno

RETORNO ACUMULADO

In [ ]:
retorno_acumulado = (1 + retorno).cumprod()
retorno_acumulado.iloc[0] = 1
retorno_acumulado

# **Gráfico comparativo**

In [ ]:
# Criar o gráfico comparativo
fig = go.Figure()
fig.add_trace(go.Scatter(x=retorno_acumulado.index, y=retorno_acumulado[symbol], name=symbol))
fig.add_trace(go.Scatter(x=retorno_acumulado.index, y=retorno_acumulado["SP500"], name="SP500"))

fig.update_layout(
    title="Retornos Acumulados - Comparação entre {} e SP500".format(symbol),
    xaxis_title="Data",
    yaxis_title="Retorno Acumulado",
    legend_title="Ticker",
)

fig.show()


In [ ]:
carteira = 10000 * retorno_acumulado.iloc[:, :5]
carteira["saldo"] = carteira.sum(axis=1)
carteira["retorno"] = carteira["saldo"].pct_change()
carteira

# **RESULTADOS**

In [ ]:
pf.create_full_tear_sheet(carteira["retorno"], benchmark_rets=retorno["SP500"])

In [ ]:
fig, ax1 = plt.subplots(figsize=(22,12))
pf.plot_rolling_beta(carteira["retorno"],factor_returns=retorno["SP500"], ax=ax1)
plt.ylim((0.5,2))